In [1]:
%tensorflow_version 2.8.0
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, ReLU, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.keras.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.8.0


In [4]:
#Need only to be used with google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            #Appending activity class to each row in the dataset
            dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            return grouped_rows
        except Exception as e:
            print(e)

In [6]:
#For long term prediction, we need a sample size of 60(10 frames input sequance, 50 frames predicted sequance)
sampled_data = Dataset_Preprocessing('/content/drive/MyDrive/Colab Notebooks/H3.6csv', sample_size=60).read_dataset()

In [7]:
total_batches = sampled_data.shape[0]

In [12]:
sampled_data = sampled_data[:total_batches-(total_batches%100)]

In [13]:
def split_to_features_labels(dataset, input_sequance_size=10) :
    assert input_sequance_size < dataset.shape[1], f"input sequance should be smaller than the total sample size"
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :64]
    
    return features, labels

In [14]:
sampled_dataX, sampled_dataY = split_to_features_labels(sampled_data, input_sequance_size=10)

In [15]:
print('Total Samples: {}'.format(sampled_dataY.shape[0]))
print('Total Frames: {}'.format(sampled_dataY.shape[1]))
print('Total Features: {}'.format(sampled_dataY.shape[2]))

Total Samples: 25500
Total Frames: 50
Total Features: 64


In [9]:
#returns a list of the interpolated frames between x0 and x1
def interpolateFrames(glogen_sparse_output_predictions, batch_size=100, num_of_frames=5) :
  timesteps = glogen_sparse_output_predictions.shape[1]
  features = glogen_sparse_output_predictions.shape[2]
  interpolated_frames = np.zeros((batch_size, timesteps, num_of_frames, features))
  
  for batch in range(batch_size) :
    for t in range(glogen_sparse_output_predictions.shape[1]-1) :
      for j in range(num_of_frames) :
        X_i0 = glogen_sparse_output_predictions[batch, t]
        X_i1 = glogen_sparse_output_predictions[batch, t+1]
        
        alpha_j = j/num_of_frames
        current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
        interpolated_frames[batch, t, j] = current_frame

  return interpolated_frames 

In [16]:
class InterpolationLayer(tf.keras.layers.Layer):
    def __init__(self, num_frames=5):
        super(InterpolationLayer, self).__init__()
        self.num_frames = num_frames
       
    def interpolateFrames(self, inputs):
      batch_size = inputs.shape[0]
      timesteps = inputs.shape[1]
      features = inputs.shape[2]
      #interpolated_frames = tf.zeros([batch_size, timesteps, 0, features])
      interpolated_frames = tf.zeros([0, features])

      for batch in range(batch_size) :
        for t in range(timesteps) :
          for j in range(self.num_frames) :
            X_i0 = inputs[batch, t]
            if(t == timesteps-1) :
              X_i1 = inputs[batch, t]
            else :  
              X_i1 = inputs[batch, t+1]
            alpha_j = j/self.num_frames
            current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
            current_frame = tf.reshape(current_frame, [1, features])
            interpolated_frames = tf.concat((interpolated_frames, current_frame), axis=0)
            #interpolated_frames[batch, t, j] = current_frame

      print(interpolated_frames.shape)

      interpolated_frames = tf.reshape(interpolated_frames,
                                       [batch_size, (timesteps)*self.num_frames, features])
      return interpolated_frames

    def call(self, inputs):
      return self.interpolateFrames(inputs)

In [15]:
class GloGen(Model_):
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200, output_diminsion=64, activation='relu'):
        super(GloGen, self).__init__()
        self.encoder = LSTM(enocder_hidden_state, return_state=True, return_sequences=True)
        self.decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
        self.dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 

    def call(self, inputs):
        encoder_outputs, state_h, state_c = self.encoder(inputs)
        encoder_states = [state_h, state_c]
        output, _, _ = self.decoder(encoder_outputs, initial_state=encoder_states)
        output = self.dense_layer(output)
        return output

In [18]:
batch_size=100

In [19]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional

encoder_inputs = Input(batch_size=batch_size,shape= (10, 74))
encoder = LSTM(200, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(encoder_outputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(64, activation='relu')
decoder_outputs = decoder_dense(decoder_outputs)

#Interpolation phase before locgen network
interpolation_layer = InterpolationLayer(num_frames=5)
interpolated_frames = interpolation_layer(decoder_outputs)

#start of locgen phase  
encoder_inputs_locgen = interpolated_frames
encoder_locgen = LSTM(200, return_state=True, return_sequences=True)
encoder_outputs_locgen, state_h_locgen, state_c_locgen = encoder_locgen(encoder_inputs_locgen)

encoder_states_locgen = [state_h_locgen, state_c_locgen]

decoder_lstm_locgen = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs_locgen, _, _ = decoder_lstm_locgen(encoder_outputs_locgen,
                                     initial_state=encoder_states_locgen)
decoder_dense_locgen = Dense(64, activation='relu')
decoder_outputs_locgen = decoder_dense_locgen(decoder_outputs_locgen)

(5000, 64)


In [20]:
glolocal_model = tf.keras.Model(inputs=encoder_inputs, outputs=decoder_outputs_locgen)

In [21]:
glolocal_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
                       loss=tf.keras.losses.mean_squared_error,
                       metrics=[tf.keras.losses.mean_absolute_percentage_error])

In [22]:
glolocal_model.fit(sampled_dataX, sampled_dataY, batch_size=100, epochs=10)

Epoch 1/50
(5000, 64)
(5000, 64)
255/255 [==============================] - 475s 281ms/step - loss: 199398.0156 - mean_absolute_percentage_error: 89.0227
Epoch 2/50
255/255 [==============================] - 72s 281ms/step - loss: 132289.6406 - mean_absolute_percentage_error: 69.3423
Epoch 3/50
255/255 [==============================] - 72s 282ms/step - loss: 87301.1484 - mean_absolute_percentage_error: 52.9392
Epoch 4/50
255/255 [==============================] - 72s 283ms/step - loss: 58293.9453 - mean_absolute_percentage_error: 40.4844
Epoch 5/50
255/255 [==============================] - 73s 287ms/step - loss: 40580.2539 - mean_absolute_percentage_error: 32.0643
Epoch 6/50
255/255 [==============================] - 71s 280ms/step - loss: 30437.5391 - mean_absolute_percentage_error: 26.9430
Epoch 7/50
255/255 [==============================] - 71s 279ms/step - loss: 25061.9590 - mean_absolute_percentage_error: 24.2190
Epoch 8/50
255/255 [==============================] - 72s 281ms/s

KeyboardInterrupt: ignored